In [430]:
import pandas as pd
import numpy as np

In [431]:
parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_data.csv')

In [432]:
parts_data['Cost Per Unit'] = parts_data['Price'] - parts_data['Margin']
parts_data['Total Cost'] = parts_data['Cost Per Unit'] * abs(parts_data['Quantity'])

In [433]:
def fix_text(data_frame, column_name):
    if (column_name != 'Part Number') & (column_name != 'Category'):
        data_frame[column_name] = data_frame[column_name].str.lower().str.strip()
    else:
        data_frame[column_name] = data_frame[column_name].str.strip()
    
    return data_frame

text_cols = parts_data.select_dtypes(exclude='number').columns
for col in text_cols:
    parts_data = fix_text(parts_data, col)

In [434]:
parts_data.head()

,Unnamed: 0,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,...,Sales - 1st Qtr,Sales - 2nd Qtr,Sales - 3rd Qtr,Sales - 4th Qtr,Sales - This Year,Sales - Last Year,Quantity Ordered YTD,Quantity Received YTD,Cost Per Unit,Total Cost
0,4871,09148-04032,SHP,nut,**added from pricebook**,25.95,0.89,3.43,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2.54,0.0
1,5532,09320-04501,SHP,cushion,**added from pricebook**,25.07,0.92,3.67,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2.75,0.0
2,5573,09329-10026,SHP,cushion,**added from pricebook**,37.37,1.11,2.97,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,1.86,0.0
3,5955,10-08207-51,ACC,painted dsr jet black with,**added from pricebook**,15.99,463.00,2895.00,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0,0,2432.00,0.0
4,22481,25-08031,ACC,fx/mx brake lever jjuan,**added from pricebook**,35.01,17.50,49.99,0.0,0.0,...,0.0,1.0,0.0,0,1.0,0.0,0,43,32.49,0.0


In [435]:
parts_data.dropna(subset=['Part Number'], inplace=True)
parts_data.dropna(subset=['Price'], inplace=True)

In [436]:
def drop_negatives(data_frame, column_name, threshold=0):
    mask = data_frame[column_name] <= threshold
    if mask.any():
        data = data_frame[~mask]
        return data
    else:
        return data_frame

# Example usage
parts_data = drop_negatives(parts_data, 'Price')


In [437]:
parts_data['Obsolete'] = parts_data['Months No Sale'].apply(lambda x: 1 if x >= 12 else 0)

In [438]:
#convert things to main suppliers and other suppliers just to simplify, then one hot encode everything
supplier_cost = parts_data.groupby('Supplier Name')['Total Cost'].sum().sort_values(ascending=False)
round(supplier_cost, 2)

Supplier Name
bmw                             527144.87
polaris                         388657.78
triumph                         349799.67
ducati                          191993.31
parts canada/power twins        135997.98
                                  ...    
fred kolman's wheelsport ltd         0.00
freedom heated clothing              0.00
fullbore marketing ltd.              0.00
gamma power sports                   0.00
ztechnik                             0.00
Name: Total Cost, Length: 245, dtype: float64

In [439]:
historical_cols = ['Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months',
       'Sales Last 9 Months', 'Sales Last 12 Months', 
       'Sales Last 2 Years','Sales Last 3 Years', 
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

this_year_cols = ['Sales - Jan', 'Sales - Feb', 'Sales - Mar',
       'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
       'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec']


In [440]:
def calculate_metric(parts_data, time_frame_cols, metric, column_to_multiply, total_sales_year):
    for col in time_frame_cols:
        # Extract the time frame (e.g., 'Jan', 'Feb', etc.) from the column name
        time_frame = col.split('-')[-1].strip()

        # Create a new column name for the metric (e.g., 'Gross Profit - Jan', etc.)
        metric_col = f'{metric} - {time_frame}'

        # Calculate the metric for the current time frame
        parts_data[metric_col] = parts_data[column_to_multiply] * parts_data[col]

    parts_data[f'Total {metric}'] = parts_data[total_sales_year] * parts_data[column_to_multiply]

    return parts_data

# Calculate Revenue for historical time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, historical_cols, 'Gross Profit', 'Price', 'Sales - Last Year')

# Calculate Profits for this year time frames (assuming you have a column 'Cost' representing the cost of each part)
parts_data = calculate_metric(parts_data, this_year_cols, 'Gross Profit', 'Price', 'Sales - This Year')

In [441]:
parts_data.columns

Index(['Unnamed: 0', 'Part Number', 'Category', 'Description', 'Supplier Name',
       'Margin %', 'Margin', 'Price', 'Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years',
       'Months No Sale', 'Reorder Point', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year', 'Quantity Ordered YTD', 'Quantity Received YTD',
       'Cost Per Unit', 'Total Cost', 'Obsolete',
       'Gross Profit - Sales Last Month', 'Gross Profit - Sales Last 3 Months',
       'Gross Profit - Sales Last 6 Months',
       'Gross Profit - Sal

In [442]:
cols_to_rename = ['Gross Profit - Sales Last Month', 
       'Gross Profit - Sales Last 3 Months',
       'Gross Profit - Sales Last 6 Months',
       'Gross Profit - Sales Last 9 Months',
       'Gross Profit - Sales Last 12 Months',
       'Gross Profit - Sales Last 2 Years',
       'Gross Profit - Sales Last 3 Years',
       'Gross Profit - Sales Last 4 Years',
       'Gross Profit - Sales Last 5 Years',
       'Gross Profit - Sales Last 10 Years']

# Define a mapping of current column names to new column names
column_mapping = {col: col.replace('Sales', '') for col in cols_to_rename}

# Rename the columns using the .rename() method
parts_data.rename(columns=column_mapping, inplace=True)


In [443]:
def negative_sales(data_frame, column_names):
    """
    The reason there are negative sales numbers is to reflect inventory that was sold prior to being added correctly into the dealership ERP,
    this is common practice during busier months as volume dramatically increases and inventory can only be added to the system every so often.
    Additionally, inventory counts occur 1 - 2 times a year which is why many of the issues such as this persist across time

    Usage: provide dataframe and columns to switch and will set negative columns to their absolute value
    """

    data_frame = data_frame.copy()  # Make a copy to avoid the warning

    for col in column_names:
        if np.issubdtype(data_frame[col].dtype, np.number):  # Only apply to numeric columns
            data_frame.loc[data_frame[col] < 0, col] = abs(data_frame[col])

    return data_frame

# Usage:
column_names = ['Sales - Last Year', 
    'Sales - Jan', 'Sales - Feb', 'Sales - Mar',
    'Sales - Apr', 'Sales - May', 'Sales - Jun', 'Sales - Jul',
    'Sales - Aug', 'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec',
    'Sales - This Year', 
    'Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months',
    'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales Last 2 Years',
    'Sales Last 3 Years', 'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years']

parts_data = negative_sales(parts_data, column_names)

In [444]:
parts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101559 entries, 0 to 103358
Data columns (total 67 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      101559 non-null  int64  
 1   Part Number                     101559 non-null  object 
 2   Category                        101559 non-null  object 
 3   Description                     101559 non-null  object 
 4   Supplier Name                   101559 non-null  object 
 5   Margin %                        101559 non-null  float64
 6   Margin                          101559 non-null  float64
 7   Price                           101559 non-null  float64
 8   Quantity                        101559 non-null  float64
 9   Sales Last Month                101559 non-null  float64
 10  Sales Last 3 Months             101559 non-null  float64
 11  Sales Last 6 Months             101559 non-null  float64
 12  Sales Last 9 Mon

In [445]:
parts_data.describe()

,Unnamed: 0,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,...,Gross Profit - Mar,Gross Profit - Apr,Gross Profit - May,Gross Profit - Jun,Gross Profit - Jul,Gross Profit - Aug,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec
count,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,...,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.000000,101559.0,101559.0,101559.0,101559.0
mean,51663.008724,40.267716,81.624103,222.430482,0.561414,0.053203,0.156439,0.261041,0.676832,0.775755,...,4.892416,4.450103,5.469930,5.705775,4.843809,0.307080,0.0,0.0,0.0,0.0
std,29820.761315,312.001345,206.638692,563.648551,5.384358,1.318415,3.481379,5.594411,8.143394,9.539861,...,68.768972,66.129359,73.893541,71.431664,77.560365,11.178608,0.0,0.0,0.0,0.0
min,0.000000,-98600.000000,-3305.000000,0.010000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-540.990000,-1256.990000,-1637.220000,-2015.040000,-579.990000,-389.990000,0.0,0.0,0.0,0.0
25%,25850.500000,33.080000,7.400000,19.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,51614.000000,38.600000,26.820000,70.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,77406.500000,49.070000,88.345000,235.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,103358.000000,100.000000,15299.000000,29060.990000,554.000000,216.000000,421.000000,695.000000,850.000000,1137.000000,...,5893.990000,4000.990000,4318.990000,4199.960000,10250.990000,1009.990000,0.0,0.0,0.0,0.0


In [446]:
def update_column_with_condition(dataframe, target_column, condition_columns, replacement_column):
    mask = (dataframe[condition_columns[0]] > 0) | (dataframe[condition_columns[1]] > 0) | (dataframe[condition_columns[2]] > 0)
    dataframe[target_column] = np.where(mask, replacement_column, dataframe[target_column])
    return dataframe

cols = ['Part Number', 'Description', 'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
        'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'Total Cost', 'COGS']

parts_data = update_column_with_condition(parts_data, 'Sales - This Year',
                                                   ['Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months'],
                                                   parts_data['Sales Last 6 Months'])

# Calculate the target for updating 'Total Cost'
target = parts_data['Cost Per Unit'] * parts_data['Quantity'].abs()

# Update 'Total Cost' based on the target calculation
parts_data = update_column_with_condition(parts_data, 'Total Cost',
                                                   ['Sales Last Month', 'Sales Last 3 Months', 'Sales Last 6 Months'],
                                                   target)

In [447]:
mask = parts_data['Total Cost'] == 0.00

# Update 'Margin' column based on the condition
parts_data['Margin'] = np.where(mask, parts_data['Price'], parts_data['Margin'])

parts_data['Margin %'] = (parts_data['Margin'] / parts_data['Price']) * 100


In [456]:
# Calculate the required metrics in the parts_data DataFrame
total_cogs = round((parts_data['Cost Per Unit'] * parts_data['Sales - This Year']).sum(), 2)
total_end_inventory = round((parts_data['Price'] * parts_data['Quantity']).sum(), 2)
total_start_inventory = (total_end_inventory - parts_data['Quantity Received YTD'].sum()) + total_cogs
avg_total_inventory = round((total_start_inventory + total_end_inventory) / 2, 2)
total_turnover = total_cogs / avg_total_inventory if avg_total_inventory > 0 else 0

# Create a list of dictionaries for the tabular chart
data = [
    {
        "Metric": "Start Inventory",
        "Value": total_start_inventory
    },
    {
        "Metric": "End Inventory",
        "Value": total_end_inventory
    },
    {
        "Metric": "Average Inventory",
        "Value": avg_total_inventory
    },
    {
        "Metric": "COGS",
        "Value": total_cogs
    },
    {
        "Metric": "Turnover",
        "Value": total_turnover
    }
]

# Print the tabular chart
table = pd.DataFrame(data, columns=['Metric', 'Value'])
table

,Metric,Value
0,Start Inventory,4.519782e+06
1,End Inventory,2.999070e+06
2,Average Inventory,3.759426e+06
3,COGS,1.994616e+06
4,Turnover,5.305639e-01


In [450]:
def create_supplier_dataframe(dataframe, suppliers=None):
    """
    Create a data frame with the turnover, cogs, and average inventory for suppliers
    
    Usage: Takes 1 or more suppliers and returns a DataFrame with the corresponding values.
           The default value if no supplier is given is all the suppliers in the data set
    """
    if suppliers is None:
        suppliers = dataframe['Supplier Name'].unique()
    
    # Filter data based on suppliers
    suppliers_data = dataframe[dataframe['Supplier Name'].isin(suppliers)]
    
    # Calculate COGS, end inventory, start inventory, average inventory, and turnover
    suppliers_data['COGS'] = suppliers_data['Cost Per Unit'] * suppliers_data['Sales - This Year']
    suppliers_data['Ending Inventory'] = suppliers_data['Price'] * suppliers_data['Quantity'].abs()
    suppliers_data['Starting Inventory'] = (suppliers_data['Ending Inventory'] - suppliers_data['Quantity Received YTD']) + suppliers_data['COGS']
    suppliers_data['Average Inventory'] = (suppliers_data['Starting Inventory'] + suppliers_data['Ending Inventory']) / 2
    suppliers_data['Turnover'] = np.where(suppliers_data['Average Inventory'] <= 0, 0, suppliers_data['COGS'] / suppliers_data['Average Inventory'])
    suppliers_data.loc[suppliers_data['COGS'] == 0, 'Turnover'] = suppliers_data['Sales - This Year']

    # Group by supplier and aggregate the data
    result_df = suppliers_data.groupby(dataframe['Supplier Name']).agg({
        'Quantity': 'sum',
        'Months No Sale': 'mean',
        'Sales Last Month': 'mean',
        'Sales Last 3 Months': 'mean',
        'Sales Last 6 Months': 'mean',
        'Sales Last 9 Months': 'mean',
        'Sales Last 12 Months': 'mean',
        'Sales - This Year': 'mean',
        'Total Cost': 'mean',
        'COGS': 'sum',
        'Starting Inventory': 'sum',
        'Ending Inventory': 'sum',
        'Average Inventory': 'sum',
        'Turnover': 'mean'
    }).reset_index()

    return result_df
  
supplier_df = create_supplier_dataframe(parts_data)

In [451]:
supplier_df

,Supplier Name,Quantity,Months No Sale,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales - This Year,Total Cost,COGS,Starting Inventory,Ending Inventory,Average Inventory,Turnover
0,**added from pricebook**,0.0,14.875000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,64.98,21.98,0.00,10.990,0.250000
1,action moto internationalrcycles / barnes,26.0,65.766478,0.0,0.002825,0.014124,0.016949,0.017891,0.016008,0.531186,414.01,1245.22,833.21,1039.215,0.027396
2,adaptive technologies,0.0,84.692308,0.0,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.00,0.00,0.00,0.000,0.000000
3,admore lighting inc.,0.0,7.000000,0.0,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00,0.00,0.00,0.000,0.000000
4,admore lights,2.0,7.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,199.000000,0.00,599.98,599.98,599.980,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,yamaha,0.0,8.232432,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000,0.000000
241,yoshimura online,0.0,30.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000,0.000000
242,zero moto internationalrcycles,-11.0,14.628959,0.0,0.004525,0.217195,0.248869,0.276018,0.366516,33.597647,11605.49,18556.99,10466.50,14511.745,0.345902
243,zmoto international,0.0,40.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000,0.000000


In [452]:
def create_part_dataframe(dataframe, part_numbers=None):
    if part_numbers is None:
        part_numbers = dataframe['Part Number'].unique()
    
    part_data = dataframe[dataframe['Part Number'].isin(part_numbers)].copy()
    
    part_data['COGS'] = part_data['Cost Per Unit'] * part_data['Sales - This Year']
    part_data['Ending Inventory'] = part_data['Price'] * part_data['Quantity'].abs()
    part_data['Starting Inventory'] = (part_data['Ending Inventory'] - part_data['Quantity Received YTD']) + part_data['COGS']
    part_data['Average Inventory'] = (part_data['Starting Inventory'] + part_data['Ending Inventory']) / 2
    part_data['Turnover'] = np.where((part_data['COGS'] == 0) & (part_data['Quantity'] != 0),
                                    part_data['Sales - This Year'],
                                    part_data['COGS'] / part_data['Average Inventory'])
    part_data['Turnover'] = part_data.loc[part_data['COGS'] >= 0, 'Turnover'] = (part_data['COGS'] / part_data['Average Inventory'])
    part_data.loc[part_data['Average Inventory'] == 0, 'Turnover'] = 0
    
    result_df = part_data.groupby(dataframe.columns.tolist()).agg({
        'COGS': 'sum',
        'Starting Inventory': 'sum',
        'Ending Inventory': 'sum',
        'Average Inventory': 'sum',
        'Turnover': 'mean'
    }).reset_index()

    #result_df['Turnover'] = np.where(result_df['Turnover'] =)
    
    return result_df

complete_parts_data = create_part_dataframe(parts_data)

In [453]:
def convert_type(dataframe, column_names, float_columns):
    for col in column_names:
        if col in float_columns:
            dataframe[col] = dataframe[col].astype('float64')
        elif dataframe[col].dtype != 'object':
            if (all(dataframe[col] % 1 == 0)) and (all(dataframe[col] >= -127) and all(dataframe[col] <= 127)):
                dataframe[col] = dataframe[col].astype('int8')
            else:
                dataframe[col] = dataframe[col].astype('int64')
        else:
            try:
                dataframe[col] = dataframe[col].astype('float64')
            except ValueError:
                pass  # Skip columns that can't be converted to float
    return dataframe


float_columns = ['Margin %','Margin', 'Price', 'Gross Profit -  Last Month',
       'Gross Profit -  Last 3 Months', 'Gross Profit -  Last 6 Months',
       'Gross Profit -  Last 9 Months', 'Gross Profit -  Last 12 Months',
       'Gross Profit -  Last 2 Years', 'Gross Profit -  Last 3 Years',
       'Gross Profit -  Last 4 Years', 'Gross Profit -  Last 5 Years',
       'Gross Profit -  Last 10 Years', 'Total Gross Profit',
       'Gross Profit - Jan', 'Gross Profit - Feb', 'Gross Profit - Mar',
       'Gross Profit - Apr', 'Gross Profit - May', 'Gross Profit - Jun',
       'Gross Profit - Jul', 'Gross Profit - Aug', 'Gross Profit - Sep',
       'Gross Profit - Oct', 'Gross Profit - Nov', 'Gross Profit - Dec',
       'Turnover', 'COGS', 'Cost Per Unit', 'Total Cost']

columns = ['Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales Last 2 Years', 'Sales Last 3 Years',
       'Sales Last 4 Years', 'Sales Last 5 Years', 'Sales Last 10 Years',
       'Months No Sale', 'Reorder Point', 'Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year', 'Quantity Ordered YTD', 'Quantity Received YTD',
       'Obsolete']
complete_parts_data = convert_type(complete_parts_data, columns, float_columns)

In [454]:
cost_mask = (complete_parts_data['Total Cost'] == 0) & (complete_parts_data['Months No Sale'] < 12)

cols = ['Part Number', 'Description', 
                    'Supplier Name', 'Price', 'Quantity', 'Months No Sale', 'Sales Last Month', 'Sales Last 3 Months', 
                    'Sales Last 6 Months', 'Sales Last 9 Months', 'Sales Last 12 Months', 'Sales - This Year', 'Turnover', 'Total Cost']

complete_parts_data[cost_mask][cols]

,Part Number,Description,Supplier Name,Price,Quantity,Months No Sale,Sales Last Month,Sales Last 3 Months,Sales Last 6 Months,Sales Last 9 Months,Sales Last 12 Months,Sales - This Year,Turnover,Total Cost
0,0,return spring,triumph,7.26,0,0,0,0,0,0,0,0,0.0,0.0
1,0 580 463 999,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,629.99,0,0,0,0,0,0,0,0,0.0,0.0
2,00-0450,"screw, gearbox,inspec&",moto international,1.95,0,0,0,0,0,0,0,0,0.0,0.0
4,00-F01740401,turn signal light control,moto international,166.99,0,0,0,0,0,0,0,0,0.0,0.0
5,00-F02600371,1399 belt drive,moto international,51.99,0,0,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101544,mv,did tip for km500 cutter,moto international,15.99,0,0,0,0,0,0,0,0,0.0,0.0
101547,s-bm0112,r1100gs 93-99 front/rear 5 lines abs2,spiegler,289.99,0,8,0,0,0,1,1,0,0.0,0.0
101554,t2048903,"yoke assy, lower",savage cycles,392.99,0,10,0,0,0,0,0,0,0.0,0.0
101555,t2501733,ytx12-bs,savage cycles,100.99,0,8,0,0,0,1,1,0,0.0,0.0


In [457]:
complete_parts_data.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/cleaned_parts_data.csv', index=False)